In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#表示の幅を広げる
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

df0 = pd.read_csv('出張買取実績_20200819以前.csv', low_memory=False)
#kaitori.head()

## "kaitori"内の姓名(カナ)が半角なのでそれを全角に変える
#### カナを軸に結合するため修正が必要

In [2]:
#Gitから日本語変換できるライブラリをインストール
!python3.8 -m pip install --upgrade pip
!pip install jaconv
import jaconv

  Using cached pip-20.3.2-py2.py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 1.5 MB 3.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached pip-20.3.1-py2.py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 1.5 MB 8.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.5 MB 505 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 5.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.5 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 3.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing 

In [3]:
#一旦、dictにいれる
L = []
for i in df0['姓名（カナ）']:
    a = jaconv.h2z(i)
    L.append(a)

#'カナ'というカラムを一旦作る
df0['カナ'] = '名前'
df0.head()

for i in range(len(L)):
   df0['カナ'][i] = L[i]
df0.head()

<ipython-input-3-b92a699237c2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0['カナ'][i] = L[i]


,仕入番号,買取日付,点数,金額,会員番号,氏名,姓名（カナ）,郵便番号,都道府県,住所1,住所2,住所3,電話番号,性別,生年月日,カナ
0,4300016249,2020/04/04,2,7000,'5000000000186206,黒川 恵美,ｸﾛｶﾜ ｴﾐ,104-0051,東京都,中央区佃1-5-11,NaN,NaN,NaN,女性,1953/10/17,クロカワ エミ
1,6600000004,2020/06/13,4,245000,'5000000000187587,高嶺 小百合,ﾀｶﾐﾈ ｻﾕﾘ,102-0082,東京都,千代田区一番町9-10,NaN,NaN,NaN,女性,1961/12/20,タカミネ サユリ
2,6600000028,2020/07/03,1,2200,'5000000000187550,山室 まつ,ﾔﾏﾑﾛ ﾏﾂ,263-0013,千葉県,千葉市稲毛区千草台,2-2-25-507,NaN,NaN,女性,1945/06/20,ヤマムロ マツ
3,6600000043,2020/07/08,2,71500,'5000000000187536,鈴木 理夫,ｽｽﾞｷ ﾏｻｵ,166-0003,東京都,杉並区高円寺南,5-24-11,NaN,NaN,女性,1940/06/04,スズキ マサオ
4,6600000051,2020/07/12,5,34100,'5000000000187528,吉田 光子,ﾖｼﾀﾞ ﾐﾂｺ,112-0002,東京都,文京区小石川,5-40-1,グリーンハウス401,NaN,女性,1969/10/02,ヨシダ ミツコ


In [4]:
#住所・市町村のカラム内で欠損している行を削除して、df1にいれる
df0 = df0.dropna(subset=['都道府県','住所1'])

In [5]:
#東京都とそれ以外で分ける
data_tokyo = df0[df0['都道府県'].str.contains('東京都')]
data1 = data_tokyo[data_tokyo['住所1'].str.contains('区')] #東京23区を抽出
data_others = df0[~df0['住所1'].str.contains('区')] #東京23区以外を抽出

#東京都23区について
#区で分けて、それを元のdfにいれる
new = data1['住所1'] .str.split("区", n = 1, expand = True) 
new[0] = new[0]+"区"
data1['住所1'] = new[0]
data1['住所2'] = new[1]

#必要な情報のみを抽出
on = ['点数', '金額','住所1']
data2 = data1[on]
data3 = data2.groupby('住所1').sum()
df3 = pd.DataFrame(data3)

<ipython-input-5-dc9b2a7cf672>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['住所1'] = new[0]
<ipython-input-5-dc9b2a7cf672>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['住所2'] = new[1]


# 住宅情報(2015年)

In [98]:
!pip install mojimoji
import mojimoji

In [190]:
kokuzei0 = pd.read_csv('2015国政調査・小地域集計.csv', low_memory=False)
kokuzei0 = kokuzei0[kokuzei0["市区町村名"].str.endswith('区')]
    
#"丁"を削除し数値のみにする
new = kokuzei0['字・丁目名'] .str.split("丁", n = 1, expand = True) 

#欠損値を0に置換
new.fillna({0:0}, inplace=True) 
kokuzei1["字・丁目名"] = new[0]

#int型をstr型に変換し、全角数字を半角数字にする
kokuzei1['字・丁目名'] = kokuzei1['字・丁目名'].astype('str')
kokuzei1['字・丁目名'] = kokuzei1["字・丁目名"].apply(mojimoji.zen_to_han)
#必要な項目のみ抽出
on = ["都道府県名","市区町村名","大字・町名","字・丁目名","一般世帯総数（住居の種類・住宅の所有の関係）","住宅に住む一般世帯","主世帯","持ち家","公営・都市再生機構・公社の借家",
      "民営の借家","給与住宅","住宅に住む一般世帯.1","主世帯.1","持ち家.1","公営・都市再生機構・公社の借家.1","民営の借家.1","給与住宅.1","間借り.1","住宅以外に住む一般世帯.1"]
kokuzei2 = kokuzei1[on]

ValueError: Unknown format code 'f' for object of type 'str'

In [185]:
kokuzei2[kokuzei2["大字・町名"] == "白金台"]

,都道府県名,市区町村名,大字・町名,字・丁目名,一般世帯総数（住居の種類・住宅の所有の関係）,住宅に住む一般世帯,主世帯,持ち家,公営・都市再生機構・公社の借家,民営の借家,給与住宅,住宅に住む一般世帯.1,主世帯.1,持ち家.1,公営・都市再生機構・公社の借家.1,民営の借家.1,給与住宅.1,間借り.1,住宅以外に住む一般世帯.1
394,東京都,港区,白金台,0,5674,5628,5583,3355,-,2014,214,11186,11111,7509,-,3190,412,75,54
395,東京都,港区,白金台,1,499,498,491,302,-,165,24,1079,1070,722,-,304,44,9,1
396,東京都,港区,白金台,2,1349,1316,1304,818,-,435,51,2654,2628,1891,-,644,93,26,34
397,東京都,港区,白金台,3,1742,1739,1733,1200,-,482,51,3664,3655,2647,-,873,135,9,4
398,東京都,港区,白金台,4,1144,1140,1125,553,-,524,48,2042,2018,1226,-,725,67,24,4
399,東京都,港区,白金台,5,940,935,930,482,-,408,40,1747,1740,1023,-,644,73,7,11


## 世帯所得情報（2015）

In [135]:
TYO_income = pd.read_csv('東京都世帯収入2015.csv', low_memory=False) #from e-stat
TYO_income.rename(columns={'市区町村':'市区町村名'}, inplace = True) #ラベル名変更
TYO_income.head(3)

,ID,県,市区町村名,普通世帯総数,25歳未満,25〜34歳,35〜44,45〜54,55〜64,65歳以上,自営業主,農林・漁業業主,商工・その他の業主,雇用者,会社・団体・公社又は個人に雇われている者,官公庁,労働者派遣事業所の派遣社員,パート・アルバイト・その他,無職,学生,その他,300万円未満,300〜500,500〜700,700〜1000,1000万円以上
0,100,東京都,特別区部,4613570,160940,548720,736050,712320,649570,1240920,370710,2920,367790,1494560,1102490,83290,57170,251610,623290,36500,586790,1336500,1105980,664560,521910,439290
1,101,東京都,千代田区,27590,530,4640,4720,4610,4460,6850,3050,0,3050,8390,5370,2400,130,480,2370,210,2160,3540,5290,5950,4910,6170
2,102,東京都,中央区,76170,2170,13110,16270,15270,10320,15060,6390,60,6340,27950,23600,1320,630,2400,6370,210,6170,13750,17490,13430,12460,15320


### 高級住宅街のリスト

In [193]:
dff = pd.read_csv('エリア(区)ごとの買取総額 - 高級持ち家エリア.csv', low_memory=False)
dff["市区町村名"] = dff["市区町村名"] + "区"

#必要な要素のみ抽出
on = ["市区町村名","大字・町名","字・丁目名"]
rich_nbhd = dff[on]

#欠損値の補完
rich_nbhd.fillna({'字・丁目名':0}, inplace=True)
rich_nbhd = rich_nbhd.astype(str)
rich_nbhd.dtypes

/Users/satoshiido/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


市区町村名    object
大字・町名    object
字・丁目名    object
dtype: object

## 各dfを結合させる 

In [170]:
df0 = pd.merge(rich_nbhd, TYO_income, on='市区町村名', how='left')

In [192]:
df1 = pd.merge(df0, kokuzei2, on = ["市区町村名", "大字・町名", "字・丁目名"], how='left')
df1

,市区町村名,大字・町名,字・丁目名,ID,県,普通世帯総数,25歳未満,25〜34歳,35〜44,45〜54,55〜64,65歳以上,自営業主,農林・漁業業主,商工・その他の業主,雇用者,会社・団体・公社又は個人に雇われている者,官公庁,労働者派遣事業所の派遣社員,パート・アルバイト・その他,無職,学生,その他,300万円未満,300〜500,500〜700,700〜1000,1000万円以上,都道府県名,一般世帯総数（住居の種類・住宅の所有の関係）,住宅に住む一般世帯,主世帯,持ち家,公営・都市再生機構・公社の借家,民営の借家,給与住宅,住宅に住む一般世帯.1,主世帯.1,持ち家.1,公営・都市再生機構・公社の借家.1,民営の借家.1,給与住宅.1,間借り.1,住宅以外に住む一般世帯.1
0,品川区,東五反田,1.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,品川区,東五反田,3.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,品川区,東五反田,4.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,品川区,東五反田,5.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,品川区,北品川,3.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,品川区,北品川,4.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,品川区,北品川,5.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,品川区,北品川,6.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,品川区,北品川,4.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,品川区,上大崎,3.0,109,東京都,198710,6420,26790,37340,29030,28590,52240,15270,80,15190,73440,57820,2050,2660,10910,27270,800,26460,52130,47130,30550,28000,23770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
